# Advanced Climate Emulators: 1D CNN & Spherical FNO

This notebook implements advanced architectures for climate emulation:

1. **1D CNN (Vertical Profile)** - Convolutional layers along the vertical dimension
2. **Stochastic Output Head** - Uncertainty quantification via parameterized Gaussian
3. **Per-Variable-Group R² Monitoring** - Separate metrics for temperature, moisture, clouds
4. **Multi-GPU Training** - JAX pmap/shmap for data parallelism
5. **Spherical FNO** - Reference implementation (advanced)

## Why 1D CNN?

- **Vertical Structure**: Atmospheric columns have clear vertical structure
- **Local Patterns**: Physics often involves local vertical interactions
- **Inductive Bias**: CNNs naturally capture spatial hierarchies
- **Parameter Efficiency**: Fewer parameters than fully-connected layers

## Why Stochastic Head?

- **Uncertainty**: Real atmosphere is stochastic
- **Ensemble Emulation**: Capture distribution of possible outcomes
- **Risk Assessment**: Important for extreme events

## Setup & Imports

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
from jax import random, grad, jit, vmap, pmap
from jax.experimental import mesh_utils
from jax.sharding import Mesh, PartitionSpec, NamedSharding
import optax
import flax.linen as nn
from flax.training import train_state, checkpoints
import orbax.checkpoint as ocp
from pathlib import Path
import os
from typing import Any, Callable, Sequence, Optional, Tuple
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
import time
from functools import partial

# Try to import torch-harmonics (for SFNO)
try:
    import torch
    import torch_harmonics as th
    from torch_harmonics import RealSHT, InverseRealSHT
    TORCH_HARMONICS_AVAILABLE = True
    print(f"✓ torch-harmonics available (version: {th.__version__ if hasattr(th, '__version__') else 'unknown'})")
except ImportError:
    TORCH_HARMONICS_AVAILABLE = False
    print("⚠ torch-harmonics not available. SFNO will not be functional.")
    print("  Install with: pip install torch-harmonics")

print(f"JAX version: {jax.__version__}")
print(f"JAX devices: {jax.devices()}")
print(f"Number of devices: {jax.device_count()}")

## Configuration

In [ ]:
@dataclass
class Config:
    # Data paths
    DATA_DIR: Path = Path("/home/jovyan/leap-scratch") / os.environ.get("USER", "user") / "data"
    MODEL_DIR: Path = Path("/home/jovyan/leap-scratch") / os.environ.get("USER", "user") / "models" / "advanced"
    
    # Model architecture
    MODEL_TYPE: str = "cnn1d"  # "cnn1d", "sfno" - change to "sfno" to use Spherical FNO
    
    # 1D CNN architecture
    CNN_CHANNELS: Sequence[int] = (64, 128, 256, 128, 64)  # Channel progression
    CNN_KERNEL_SIZE: int = 3  # Kernel size for conv layers
    USE_RESIDUAL: bool = True  # Use residual connections
    USE_BATCH_NORM: bool = False  # Use batch norm (GroupNorm better for small batches)
    USE_GROUP_NORM: bool = True  # Use group norm
    NUM_GROUPS: int = 8  # Number of groups for GroupNorm
    
    # Stochastic output
    STOCHASTIC_OUTPUT: bool = True  # Enable uncertainty quantification
    MIN_STD: float = 1e-4  # Minimum std to prevent collapse
    
    # SFNO architecture (if MODEL_TYPE == "sfno")
    SFNO_MODES: int = 16  # Number of Fourier modes
    SFNO_WIDTH: int = 256  # Hidden dimension
    SFNO_LAYERS: int = 4  # Number of FNO layers
    
    # Training
    BATCH_SIZE: int = 128  # Per-device batch size
    NUM_EPOCHS: int = 50
    LEARNING_RATE: float = 1e-3
    WEIGHT_DECAY: float = 1e-5
    WARMUP_EPOCHS: int = 5
    DROPOUT_RATE: float = 0.1
    
    # Loss weights
    MSE_WEIGHT: float = 1.0
    NLL_WEIGHT: float = 0.1  # Negative log-likelihood for stochastic output
    WATER_CONS_WEIGHT: float = 0.0  # Optional water conservation
    
    # Multi-GPU
    USE_PMAP: bool = jax.device_count() > 1
    
    # Variable groups for monitoring
    VAR_GROUPS: dict = field(default_factory=lambda: {
        'temperature': ['ptend_t'],
        'moisture': ['ptend_q0001', 'ptend_q0002', 'ptend_q0003'],
        'clouds': ['ptend_u', 'ptend_v'],  # Adjust based on actual variables
    })
    
    # Checkpoint
    SAVE_EVERY: int = 5
    
    def __post_init__(self):
        self.DATA_DIR.mkdir(parents=True, exist_ok=True)
        self.MODEL_DIR.mkdir(parents=True, exist_ok=True)

config = Config()
print(f"Config: {config.MODEL_TYPE.upper()} architecture")
print(f"Multi-GPU: {config.USE_PMAP} ({jax.device_count()} devices)")
print(f"Stochastic output: {config.STOCHASTIC_OUTPUT}")

## Load Preprocessed Data

We'll load the preprocessed data from the previous notebook. For 1D CNN, we need to reshape the data to include a spatial dimension (vertical levels).

In [ ]:
# Load preprocessed data
data_path = config.DATA_DIR / "climsim_nyc_processed.npz"

if data_path.exists():
    print(f"Loading data from {data_path}")
    data = np.load(data_path)
    X_train = data['X_train']
    y_train = data['y_train']
    X_val = data['X_val']
    y_val = data['y_val']
    X_test = data['X_test']
    y_test = data['y_test']
    input_mean = data['input_mean']
    input_std = data['input_std']
    output_mean = data['output_mean']
    output_std = data['output_std']
    print(f"✓ Loaded {X_train.shape[0]} training samples")
else:
    print("⚠ Preprocessed data not found. Generating synthetic data for demonstration...")
    # Generate synthetic data
    n_train, n_val, n_test = 8000, 1000, 1000
    n_levels = 60  # Typical vertical levels
    n_vars = 5  # Variables per level (T, q, u, v, etc.)
    input_dim = n_levels * n_vars  # ~300
    output_dim = n_levels * 3  # Temperature, moisture, momentum tendencies
    
    X_train = np.random.randn(n_train, input_dim).astype(np.float32)
    y_train = np.random.randn(n_train, output_dim).astype(np.float32) * 0.1
    X_val = np.random.randn(n_val, input_dim).astype(np.float32)
    y_val = np.random.randn(n_val, output_dim).astype(np.float32) * 0.1
    X_test = np.random.randn(n_test, input_dim).astype(np.float32)
    y_test = np.random.randn(n_test, output_dim).astype(np.float32) * 0.1
    
    input_mean = np.zeros(input_dim, dtype=np.float32)
    input_std = np.ones(input_dim, dtype=np.float32)
    output_mean = np.zeros(output_dim, dtype=np.float32)
    output_std = np.ones(output_dim, dtype=np.float32)
    print(f"✓ Generated synthetic data")

print(f"\nData shapes:")
print(f"  Train: X={X_train.shape}, y={y_train.shape}")
print(f"  Val:   X={X_val.shape}, y={y_val.shape}")
print(f"  Test:  X={X_test.shape}, y={y_test.shape}")

## Reshape Data for 1D CNN

CNNs expect spatial dimensions. We'll reshape flattened vectors into (levels, channels) format:
- Input: (batch, features) → (batch, levels, channels)
- Output: (batch, features) → (batch, levels, output_channels)

In [ ]:
# Infer structure from data dimensions
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

# Common vertical levels in climate models
possible_levels = [26, 30, 60, 72, 91]
n_levels = None

for levels in possible_levels:
    if input_dim % levels == 0:
        n_levels = levels
        n_input_vars = input_dim // levels
        break

if n_levels is None:
    # Default: assume 60 levels
    n_levels = 60
    n_input_vars = input_dim // n_levels
    if input_dim % n_levels != 0:
        # Pad to make it divisible
        pad_size = n_levels - (input_dim % n_levels)
        X_train = np.pad(X_train, ((0, 0), (0, pad_size)), mode='constant')
        X_val = np.pad(X_val, ((0, 0), (0, pad_size)), mode='constant')
        X_test = np.pad(X_test, ((0, 0), (0, pad_size)), mode='constant')
        input_dim = X_train.shape[1]
        n_input_vars = input_dim // n_levels

# Same for output
if output_dim % n_levels == 0:
    n_output_vars = output_dim // n_levels
else:
    pad_size = n_levels - (output_dim % n_levels)
    y_train = np.pad(y_train, ((0, 0), (0, pad_size)), mode='constant')
    y_val = np.pad(y_val, ((0, 0), (0, pad_size)), mode='constant')
    y_test = np.pad(y_test, ((0, 0), (0, pad_size)), mode='constant')
    output_dim = y_train.shape[1]
    n_output_vars = output_dim // n_levels

print(f"\nInferred structure:")
print(f"  Vertical levels: {n_levels}")
print(f"  Input variables per level: {n_input_vars}")
print(f"  Output variables per level: {n_output_vars}")

# Reshape: (batch, features) -> (batch, levels, vars_per_level)
def reshape_for_cnn(X, n_levels, n_vars):
    """Reshape flattened data to (batch, levels, channels)"""
    batch_size = X.shape[0]
    return X.reshape(batch_size, n_levels, n_vars)

X_train_cnn = reshape_for_cnn(X_train, n_levels, n_input_vars)
X_val_cnn = reshape_for_cnn(X_val, n_levels, n_input_vars)
X_test_cnn = reshape_for_cnn(X_test, n_levels, n_input_vars)

y_train_cnn = reshape_for_cnn(y_train, n_levels, n_output_vars)
y_val_cnn = reshape_for_cnn(y_val, n_levels, n_output_vars)
y_test_cnn = reshape_for_cnn(y_test, n_levels, n_output_vars)

print(f"\nReshaped for CNN:")
print(f"  X_train: {X_train_cnn.shape} (batch, levels, input_channels)")
print(f"  y_train: {y_train_cnn.shape} (batch, levels, output_channels)")

## 1D CNN Architecture

We'll build a 1D CNN with:
- Residual connections (inspired by ResNet)
- Group normalization (better than batch norm for small batches)
- Stochastic output head (mean + log_std)

In [ ]:
class ResidualBlock1D(nn.Module):
    """1D Residual block with optional normalization"""
    channels: int
    kernel_size: int = 3
    use_group_norm: bool = True
    num_groups: int = 8
    dropout_rate: float = 0.1
    
    @nn.compact
    def __call__(self, x, training: bool = False):
        residual = x
        
        # First conv
        x = nn.Conv(self.channels, kernel_size=(self.kernel_size,), padding='SAME')(x)
        if self.use_group_norm:
            x = nn.GroupNorm(num_groups=self.num_groups)(x)
        x = nn.swish(x)
        x = nn.Dropout(rate=self.dropout_rate, deterministic=not training)(x)
        
        # Second conv
        x = nn.Conv(self.channels, kernel_size=(self.kernel_size,), padding='SAME')(x)
        if self.use_group_norm:
            x = nn.GroupNorm(num_groups=self.num_groups)(x)
        
        # Residual connection (with projection if needed)
        if residual.shape[-1] != self.channels:
            residual = nn.Conv(self.channels, kernel_size=(1,), padding='SAME')(residual)
        
        x = nn.swish(x + residual)
        return x


class StochasticOutputHead(nn.Module):
    """Output head that predicts mean and log_std for uncertainty quantification"""
    output_dim: int
    min_std: float = 1e-4
    
    @nn.compact
    def __call__(self, x):
        # Predict mean
        mean = nn.Dense(self.output_dim, name='mean')(x)
        
        # Predict log(std) - using log ensures std > 0
        log_std = nn.Dense(self.output_dim, name='log_std')(x)
        # Clamp to prevent numerical issues
        log_std = jnp.clip(log_std, jnp.log(self.min_std), 5.0)
        
        return mean, log_std


class CNN1DClimateEmulator(nn.Module):
    """1D CNN Climate Emulator with optional stochastic output"""
    channels: Sequence[int] = (64, 128, 256, 128, 64)
    output_channels: int = 3  # Number of output variables per level
    kernel_size: int = 3
    use_residual: bool = True
    use_group_norm: bool = True
    num_groups: int = 8
    dropout_rate: float = 0.1
    stochastic_output: bool = True
    min_std: float = 1e-4
    
    @nn.compact
    def __call__(self, x, training: bool = False):
        # x shape: (batch, levels, input_channels)
        
        # Initial projection
        x = nn.Conv(self.channels[0], kernel_size=(1,), padding='SAME')(x)
        
        # Residual blocks with increasing then decreasing channels
        if self.use_residual:
            for ch in self.channels:
                x = ResidualBlock1D(
                    channels=ch,
                    kernel_size=self.kernel_size,
                    use_group_norm=self.use_group_norm,
                    num_groups=self.num_groups,
                    dropout_rate=self.dropout_rate
                )(x, training=training)
        else:
            # Simple conv layers without residual
            for ch in self.channels:
                x = nn.Conv(ch, kernel_size=(self.kernel_size,), padding='SAME')(x)
                if self.use_group_norm:
                    x = nn.GroupNorm(num_groups=self.num_groups)(x)
                x = nn.swish(x)
                x = nn.Dropout(rate=self.dropout_rate, deterministic=not training)(x)
        
        # Output projection: (batch, levels, channels) -> (batch, levels, output_channels)
        if self.stochastic_output:
            # Flatten to apply dense layers
            batch, levels, features = x.shape
            x_flat = x.reshape(batch * levels, features)
            
            # Stochastic head
            mean, log_std = StochasticOutputHead(
                output_dim=self.output_channels,
                min_std=self.min_std
            )(x_flat)
            
            # Reshape back
            mean = mean.reshape(batch, levels, self.output_channels)
            log_std = log_std.reshape(batch, levels, self.output_channels)
            
            return mean, log_std
        else:
            # Deterministic output
            x = nn.Conv(self.output_channels, kernel_size=(1,), padding='SAME')(x)
            return x, None


print("✓ 1D CNN architecture defined")

In [ ]:
# Initialize model
model = CNN1DClimateEmulator(
    channels=config.CNN_CHANNELS,
    output_channels=n_output_vars,
    kernel_size=config.CNN_KERNEL_SIZE,
    use_residual=config.USE_RESIDUAL,
    use_group_norm=config.USE_GROUP_NORM,
    num_groups=config.NUM_GROUPS,
    dropout_rate=config.DROPOUT_RATE,
    stochastic_output=config.STOCHASTIC_OUTPUT,
    min_std=config.MIN_STD
)

# Initialize parameters
rng = random.PRNGKey(42)
rng, init_rng, dropout_rng = random.split(rng, 3)

# Sample input
sample_input = jnp.ones((1, n_levels, n_input_vars))
variables = model.init({'params': init_rng, 'dropout': dropout_rng}, sample_input, training=False)
params = variables['params']

# Count parameters
param_count = sum(x.size for x in jax.tree_util.tree_leaves(params))
print(f"\n{'='*60}")
print(f"Model: 1D CNN Climate Emulator")
print(f"{'='*60}")
print(f"Parameters: {param_count:,}")
print(f"Architecture:")
print(f"  - Input: {n_levels} levels × {n_input_vars} variables")
print(f"  - Channels: {config.CNN_CHANNELS}")
print(f"  - Kernel size: {config.CNN_KERNEL_SIZE}")
print(f"  - Residual: {config.USE_RESIDUAL}")
print(f"  - Stochastic output: {config.STOCHASTIC_OUTPUT}")
print(f"  - Output: {n_levels} levels × {n_output_vars} tendencies")
print(f"{'='*60}")

# Test forward pass
mean, log_std = model.apply(variables, sample_input, training=False, rngs={'dropout': dropout_rng})
print(f"\nTest forward pass:")
print(f"  Mean shape: {mean.shape}")
if log_std is not None:
    print(f"  Log-std shape: {log_std.shape}")
    print(f"  Std range: [{jnp.exp(log_std).min():.6f}, {jnp.exp(log_std).max():.6f}]")

## Loss Functions with Uncertainty

In [ ]:
def mse_loss(pred, target):
    """Mean Squared Error"""
    return jnp.mean((pred - target) ** 2)

def negative_log_likelihood(mean, log_std, target):
    """Negative log-likelihood for Gaussian distribution
    
    NLL = 0.5 * log(2π) + log_std + 0.5 * ((target - mean) / std)^2
    """
    std = jnp.exp(log_std)
    nll = 0.5 * jnp.log(2 * jnp.pi) + log_std + 0.5 * ((target - mean) / std) ** 2
    return jnp.mean(nll)

def water_conservation_loss(pred):
    """Water conservation constraint (optional)"""
    # Sum of moisture tendencies should be small
    # This is a simplified version - adjust based on actual physics
    moisture_sum = jnp.sum(pred, axis=-1)  # Sum over output variables
    return jnp.mean(moisture_sum ** 2)

def compute_loss(params, variables, batch_x, batch_y, rng, training=True):
    """Combined loss with MSE and optional NLL for stochastic output"""
    mean, log_std = model.apply(variables, batch_x, training=training, rngs={'dropout': rng})
    
    # MSE loss on mean prediction
    mse = mse_loss(mean, batch_y)
    loss = config.MSE_WEIGHT * mse
    
    # NLL loss for uncertainty
    if config.STOCHASTIC_OUTPUT and log_std is not None:
        nll = negative_log_likelihood(mean, log_std, batch_y)
        loss = loss + config.NLL_WEIGHT * nll
    else:
        nll = 0.0
    
    # Optional water conservation
    if config.WATER_CONS_WEIGHT > 0:
        water_cons = water_conservation_loss(mean)
        loss = loss + config.WATER_CONS_WEIGHT * water_cons
    else:
        water_cons = 0.0
    
    metrics = {
        'loss': loss,
        'mse': mse,
        'nll': nll,
        'water_cons': water_cons
    }
    
    return loss, (mean, log_std, metrics)

print("✓ Loss functions defined")

## Per-Variable-Group R² Metrics

In [ ]:
def compute_r2_score(pred, target):
    """Compute R² score"""
    ss_res = jnp.sum((target - pred) ** 2)
    ss_tot = jnp.sum((target - jnp.mean(target)) ** 2)
    r2 = 1 - (ss_res / (ss_tot + 1e-8))
    return r2

def compute_per_variable_r2(pred, target, var_groups, output_dim):
    """Compute R² for each variable group
    
    Note: This is a simplified version. In practice, you'd map variable names
    to output dimensions based on your data structure.
    """
    # Flatten spatial dimensions for per-variable metrics
    pred_flat = pred.reshape(-1, output_dim)
    target_flat = target.reshape(-1, output_dim)
    
    results = {}
    
    # For demonstration, assume variables are ordered:
    # [temp_vars, moisture_vars, cloud_vars]
    # Adjust based on actual data structure
    
    # Example: first 1/3 are temperature, second 1/3 moisture, last 1/3 clouds
    n_per_group = output_dim // 3
    
    groups = {
        'temperature': (0, n_per_group),
        'moisture': (n_per_group, 2 * n_per_group),
        'clouds': (2 * n_per_group, output_dim)
    }
    
    for group_name, (start, end) in groups.items():
        if end > start:
            pred_group = pred_flat[:, start:end]
            target_group = target_flat[:, start:end]
            r2 = compute_r2_score(pred_group, target_group)
            results[f'r2_{group_name}'] = r2
    
    # Overall R²
    results['r2_overall'] = compute_r2_score(pred_flat, target_flat)
    
    return results

print("✓ Per-variable R² metrics defined")

## Training Setup

In [ ]:
# Learning rate schedule
steps_per_epoch = len(X_train_cnn) // config.BATCH_SIZE
total_steps = steps_per_epoch * config.NUM_EPOCHS
warmup_steps = steps_per_epoch * config.WARMUP_EPOCHS

lr_schedule = optax.warmup_cosine_decay_schedule(
    init_value=1e-8,
    peak_value=config.LEARNING_RATE,
    warmup_steps=warmup_steps,
    decay_steps=total_steps,
    end_value=1e-6
)

# Optimizer
optimizer = optax.adamw(
    learning_rate=lr_schedule,
    weight_decay=config.WEIGHT_DECAY
)

# Train state
class TrainState(train_state.TrainState):
    dropout_rng: jax.random.PRNGKey

state = TrainState.create(
    apply_fn=model.apply,
    params=params,
    tx=optimizer,
    dropout_rng=dropout_rng
)

print(f"\nTraining setup:")
print(f"  Total steps: {total_steps}")
print(f"  Warmup steps: {warmup_steps}")
print(f"  Steps per epoch: {steps_per_epoch}")
print(f"  Learning rate: {config.LEARNING_RATE}")

## Training & Evaluation Steps

In [ ]:
# Training step (JIT compiled)
@jit
def train_step(state, batch_x, batch_y):
    """Single training step"""
    dropout_rng, new_dropout_rng = random.split(state.dropout_rng)
    
    def loss_fn(params):
        variables = {'params': params}
        return compute_loss(params, variables, batch_x, batch_y, dropout_rng, training=True)
    
    (loss, (mean, log_std, metrics)), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params)
    state = state.apply_gradients(grads=grads)
    state = state.replace(dropout_rng=new_dropout_rng)
    
    return state, metrics

# Evaluation step (JIT compiled)
@jit
def eval_step(state, batch_x, batch_y):
    """Single evaluation step"""
    variables = {'params': state.params}
    loss, (mean, log_std, metrics) = compute_loss(
        state.params, variables, batch_x, batch_y, state.dropout_rng, training=False
    )
    return mean, log_std, metrics

print("✓ Training and evaluation steps defined (JIT compiled)")

## Multi-GPU Data Parallel Training (Optional)

If multiple GPUs are available, we'll use `pmap` for data parallelism.

In [ ]:
if config.USE_PMAP:
    print(f"\n{'='*60}")
    print(f"Multi-GPU Training Setup")
    print(f"{'='*60}")
    print(f"Devices: {jax.device_count()}")
    
    # Replicate state across devices
    state = jax.device_put_replicated(state, jax.devices())
    
    # Define pmapped training step
    @partial(pmap, axis_name='batch')
    def train_step_pmap(state, batch_x, batch_y):
        """Pmapped training step"""
        dropout_rng, new_dropout_rng = random.split(state.dropout_rng)
        
        def loss_fn(params):
            variables = {'params': params}
            return compute_loss(params, variables, batch_x, batch_y, dropout_rng, training=True)
        
        (loss, (mean, log_std, metrics)), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params)
        
        # Average gradients across devices
        grads = jax.lax.pmean(grads, axis_name='batch')
        
        state = state.apply_gradients(grads=grads)
        state = state.replace(dropout_rng=new_dropout_rng)
        
        # Average metrics across devices
        metrics = jax.tree_map(lambda x: jax.lax.pmean(x, axis_name='batch'), metrics)
        
        return state, metrics
    
    @partial(pmap, axis_name='batch')
    def eval_step_pmap(state, batch_x, batch_y):
        """Pmapped evaluation step"""
        variables = {'params': state.params}
        loss, (mean, log_std, metrics) = compute_loss(
            state.params, variables, batch_x, batch_y, state.dropout_rng, training=False
        )
        metrics = jax.tree_map(lambda x: jax.lax.pmean(x, axis_name='batch'), metrics)
        return mean, log_std, metrics
    
    # Use pmap versions
    train_step_fn = train_step_pmap
    eval_step_fn = eval_step_pmap
    
    print("✓ Multi-GPU training enabled (pmap)")
else:
    # Single device
    train_step_fn = train_step
    eval_step_fn = eval_step
    print("Single-GPU training (no pmap)")

## Data Loader

In [ ]:
def create_batches(X, y, batch_size, shuffle=True, num_devices=1):
    """Create batches, optionally sharded across devices"""
    n_samples = len(X)
    indices = np.arange(n_samples)
    
    if shuffle:
        np.random.shuffle(indices)
    
    # Adjust batch size for multi-device
    total_batch_size = batch_size * num_devices
    
    # Drop last incomplete batch
    n_batches = n_samples // total_batch_size
    indices = indices[:n_batches * total_batch_size]
    
    for i in range(0, len(indices), total_batch_size):
        batch_idx = indices[i:i + total_batch_size]
        batch_x = X[batch_idx]
        batch_y = y[batch_idx]
        
        if num_devices > 1:
            # Reshape for pmap: (num_devices, per_device_batch_size, ...)
            batch_x = batch_x.reshape(num_devices, -1, *batch_x.shape[1:])
            batch_y = batch_y.reshape(num_devices, -1, *batch_y.shape[1:])
        
        yield jnp.array(batch_x), jnp.array(batch_y)

print("✓ Data loader defined")

## Training Loop

This is a simplified training loop. Set `config.NUM_EPOCHS` to a lower value (e.g., 5-10) for quick testing, or increase to 50+ for full training.

In [ ]:
# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'train_mse': [],
    'val_mse': [],
    'train_nll': [],
    'val_nll': [],
    'val_r2_temperature': [],
    'val_r2_moisture': [],
    'val_r2_clouds': [],
    'val_r2_overall': [],
}

best_val_loss = float('inf')
best_state = None

num_devices = jax.device_count() if config.USE_PMAP else 1

print(f"\n{'='*60}")
print(f"Starting Training")
print(f"{'='*60}")
print(f"Epochs: {config.NUM_EPOCHS}")
print(f"Batch size: {config.BATCH_SIZE} × {num_devices} devices = {config.BATCH_SIZE * num_devices}")
print(f"Training samples: {len(X_train_cnn)}")
print(f"Validation samples: {len(X_val_cnn)}")
print(f"{'='*60}\n")

start_time = time.time()

for epoch in range(config.NUM_EPOCHS):
    epoch_start = time.time()
    
    # Training
    train_metrics_epoch = []
    for batch_x, batch_y in create_batches(X_train_cnn, y_train_cnn, config.BATCH_SIZE, 
                                            shuffle=True, num_devices=num_devices):
        if config.USE_PMAP:
            state, metrics = train_step_fn(state, batch_x, batch_y)
            # Extract metrics from first device
            metrics = jax.tree_map(lambda x: x[0], metrics)
        else:
            state, metrics = train_step_fn(state, batch_x, batch_y)
        train_metrics_epoch.append(metrics)
    
    # Average training metrics
    train_loss = np.mean([m['loss'] for m in train_metrics_epoch])
    train_mse = np.mean([m['mse'] for m in train_metrics_epoch])
    train_nll = np.mean([m['nll'] for m in train_metrics_epoch])
    
    # Validation
    val_metrics_epoch = []
    val_predictions = []
    val_targets = []
    
    for batch_x, batch_y in create_batches(X_val_cnn, y_val_cnn, config.BATCH_SIZE, 
                                            shuffle=False, num_devices=num_devices):
        if config.USE_PMAP:
            mean, log_std, metrics = eval_step_fn(state, batch_x, batch_y)
            # Extract from first device
            metrics = jax.tree_map(lambda x: x[0], metrics)
            mean = mean[0]  # First device
            batch_y_cpu = batch_y[0]  # First device
        else:
            mean, log_std, metrics = eval_step_fn(state, batch_x, batch_y)
            batch_y_cpu = batch_y
        
        val_metrics_epoch.append(metrics)
        val_predictions.append(np.array(mean))
        val_targets.append(np.array(batch_y_cpu))
    
    # Average validation metrics
    val_loss = np.mean([m['loss'] for m in val_metrics_epoch])
    val_mse = np.mean([m['mse'] for m in val_metrics_epoch])
    val_nll = np.mean([m['nll'] for m in val_metrics_epoch])
    
    # Compute per-variable R²
    val_predictions_all = np.concatenate(val_predictions, axis=0)
    val_targets_all = np.concatenate(val_targets, axis=0)
    r2_metrics = compute_per_variable_r2(
        val_predictions_all, val_targets_all, config.VAR_GROUPS, 
        val_predictions_all.shape[-1]
    )
    
    # Update history
    history['train_loss'].append(float(train_loss))
    history['val_loss'].append(float(val_loss))
    history['train_mse'].append(float(train_mse))
    history['val_mse'].append(float(val_mse))
    history['train_nll'].append(float(train_nll))
    history['val_nll'].append(float(val_nll))
    history['val_r2_temperature'].append(float(r2_metrics['r2_temperature']))
    history['val_r2_moisture'].append(float(r2_metrics['r2_moisture']))
    history['val_r2_clouds'].append(float(r2_metrics['r2_clouds']))
    history['val_r2_overall'].append(float(r2_metrics['r2_overall']))
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        if config.USE_PMAP:
            # Save from first device
            best_state = jax.tree_map(lambda x: x[0], state)
        else:
            best_state = state
    
    # Periodic checkpoint
    if (epoch + 1) % config.SAVE_EVERY == 0:
        ckpt_dir = config.MODEL_DIR / f"checkpoint_epoch_{epoch+1}"
        ckpt_dir.mkdir(exist_ok=True)
        
        if config.USE_PMAP:
            state_to_save = jax.tree_map(lambda x: x[0], state)
        else:
            state_to_save = state
        
        checkpoints.save_checkpoint(
            ckpt_dir=str(ckpt_dir),
            target=state_to_save,
            step=epoch + 1,
            overwrite=True
        )
    
    epoch_time = time.time() - epoch_start
    
    # Print progress
    print(f"Epoch {epoch+1:3d}/{config.NUM_EPOCHS} [{epoch_time:5.1f}s] | "
          f"Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | "
          f"R² (T/M/C): {r2_metrics['r2_temperature']:.3f}/{r2_metrics['r2_moisture']:.3f}/{r2_metrics['r2_clouds']:.3f}")

total_time = time.time() - start_time
print(f"\n{'='*60}")
print(f"Training completed in {total_time:.1f}s ({total_time/60:.1f} min)")
print(f"Best validation loss: {best_val_loss:.6f}")
print(f"{'='*60}")

## Save Best Model

In [ ]:
# Save best model with Orbax
best_model_dir = config.MODEL_DIR / "best_model"
best_model_dir.mkdir(exist_ok=True)

checkpointer = ocp.PyTreeCheckpointer()
checkpointer.save(best_model_dir / "checkpoint", best_state)

# Save config and history
np.savez(
    config.MODEL_DIR / "training_results.npz",
    **history,
    best_val_loss=best_val_loss,
    config_dict={
        'model_type': config.MODEL_TYPE,
        'cnn_channels': list(config.CNN_CHANNELS),
        'kernel_size': config.CNN_KERNEL_SIZE,
        'stochastic_output': config.STOCHASTIC_OUTPUT,
        'num_params': param_count,
    }
)

print(f"\n✓ Model saved to {best_model_dir}")
print(f"✓ Training results saved to {config.MODEL_DIR / 'training_results.npz'}")

## Visualization: Training Curves

In [ ]:
# Plot training curves
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss curves
ax = axes[0, 0]
ax.plot(history['train_loss'], label='Train', alpha=0.8)
ax.plot(history['val_loss'], label='Validation', alpha=0.8)
ax.set_xlabel('Epoch')
ax.set_ylabel('Total Loss')
ax.set_title('Training & Validation Loss')
ax.legend()
ax.grid(True, alpha=0.3)

# MSE curves
ax = axes[0, 1]
ax.plot(history['train_mse'], label='Train MSE', alpha=0.8)
ax.plot(history['val_mse'], label='Val MSE', alpha=0.8)
ax.set_xlabel('Epoch')
ax.set_ylabel('MSE')
ax.set_title('Mean Squared Error')
ax.legend()
ax.grid(True, alpha=0.3)

# R² scores by variable group
ax = axes[1, 0]
ax.plot(history['val_r2_temperature'], label='Temperature', alpha=0.8)
ax.plot(history['val_r2_moisture'], label='Moisture', alpha=0.8)
ax.plot(history['val_r2_clouds'], label='Clouds', alpha=0.8)
ax.plot(history['val_r2_overall'], label='Overall', alpha=0.8, linewidth=2, color='black')
ax.set_xlabel('Epoch')
ax.set_ylabel('R² Score')
ax.set_title('R² by Variable Group')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim([max(-1, min(history['val_r2_overall']) - 0.1), 1.0])

# NLL (if stochastic)
ax = axes[1, 1]
if config.STOCHASTIC_OUTPUT:
    ax.plot(history['train_nll'], label='Train NLL', alpha=0.8)
    ax.plot(history['val_nll'], label='Val NLL', alpha=0.8)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Negative Log-Likelihood')
    ax.set_title('Uncertainty (NLL)')
    ax.legend()
    ax.grid(True, alpha=0.3)
else:
    ax.text(0.5, 0.5, 'Stochastic output disabled', 
            ha='center', va='center', transform=ax.transAxes)
    ax.axis('off')

plt.tight_layout()
plt.savefig(config.MODEL_DIR / 'training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Training curves saved")

## Test Set Evaluation

In [ ]:
# Evaluate on test set
print(f"\n{'='*60}")
print("Test Set Evaluation")
print(f"{'='*60}\n")

test_predictions = []
test_uncertainties = []
test_targets_list = []
test_metrics = []

# Use best_state for evaluation
eval_state = best_state

for batch_x, batch_y in create_batches(X_test_cnn, y_test_cnn, config.BATCH_SIZE, 
                                        shuffle=False, num_devices=1):  # Always use 1 device for eval
    variables = {'params': eval_state.params}
    mean, log_std = model.apply(variables, batch_x, training=False, rngs={'dropout': eval_state.dropout_rng})
    
    # Compute metrics
    loss, (_, _, metrics) = compute_loss(eval_state.params, variables, batch_x, batch_y, 
                                         eval_state.dropout_rng, training=False)
    
    test_predictions.append(np.array(mean))
    if log_std is not None:
        test_uncertainties.append(np.array(jnp.exp(log_std)))
    test_targets_list.append(np.array(batch_y))
    test_metrics.append(metrics)

test_pred = np.concatenate(test_predictions, axis=0)
test_targets = np.concatenate(test_targets_list, axis=0)
if config.STOCHASTIC_OUTPUT:
    test_std = np.concatenate(test_uncertainties, axis=0)

# Compute test metrics
test_loss = np.mean([m['loss'] for m in test_metrics])
test_mse = np.mean([m['mse'] for m in test_metrics])
test_mae = np.mean(np.abs(test_pred - test_targets))
test_rmse = np.sqrt(test_mse)
test_r2_metrics = compute_per_variable_r2(test_pred, test_targets, config.VAR_GROUPS, test_pred.shape[-1])

print(f"Test Loss:       {test_loss:.6f}")
print(f"Test MSE:        {test_mse:.6f}")
print(f"Test MAE:        {test_mae:.6f}")
print(f"Test RMSE:       {test_rmse:.6f}")
print(f"\nR² Scores:")
print(f"  Overall:       {test_r2_metrics['r2_overall']:.4f}")
print(f"  Temperature:   {test_r2_metrics['r2_temperature']:.4f}")
print(f"  Moisture:      {test_r2_metrics['r2_moisture']:.4f}")
print(f"  Clouds:        {test_r2_metrics['r2_clouds']:.4f}")

if config.STOCHASTIC_OUTPUT:
    mean_uncertainty = np.mean(test_std)
    print(f"\nUncertainty (mean std): {mean_uncertainty:.6f}")

print(f"\n{'='*60}")

## Visualize Predictions & Uncertainty

In [ ]:
# Vertical profile comparison
sample_idx = np.random.randint(0, len(test_pred))

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# For each output variable, plot a vertical profile
for var_idx in range(min(3, n_output_vars)):
    ax = axes[var_idx]
    
    levels = np.arange(n_levels)
    pred_profile = test_pred[sample_idx, :, var_idx]
    target_profile = test_targets[sample_idx, :, var_idx]
    
    ax.plot(pred_profile, levels, 'b-', label='Prediction', linewidth=2)
    ax.plot(target_profile, levels, 'r--', label='Target', linewidth=2)
    
    if config.STOCHASTIC_OUTPUT:
        std_profile = test_std[sample_idx, :, var_idx]
        ax.fill_betweenx(levels, 
                         pred_profile - 2*std_profile, 
                         pred_profile + 2*std_profile,
                         alpha=0.3, color='blue', label='±2σ uncertainty')
    
    ax.set_ylabel('Vertical Level')
    ax.set_xlabel(f'Variable {var_idx+1}')
    ax.set_title(f'Vertical Profile - Output Var {var_idx+1}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.invert_yaxis()  # Top of atmosphere at top

plt.tight_layout()
plt.savefig(config.MODEL_DIR / 'vertical_profiles.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Vertical profiles saved")

## Prediction vs Target Scatter

In [ ]:
# Scatter plot: prediction vs target (flattened)
fig, ax = plt.subplots(figsize=(8, 8))

pred_flat = test_pred.flatten()
target_flat = test_targets.flatten()

# Subsample for visualization
n_plot = min(10000, len(pred_flat))
plot_idx = np.random.choice(len(pred_flat), n_plot, replace=False)

ax.scatter(target_flat[plot_idx], pred_flat[plot_idx], alpha=0.3, s=1)
ax.plot([target_flat.min(), target_flat.max()], 
        [target_flat.min(), target_flat.max()], 
        'r--', linewidth=2, label='Perfect prediction')
ax.set_xlabel('Target')
ax.set_ylabel('Prediction')
ax.set_title(f'Prediction vs Target (R²={test_r2_metrics["r2_overall"]:.3f})')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(config.MODEL_DIR / 'prediction_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Prediction scatter saved")

## Spherical Fourier Neural Operator (SFNO) with torch-harmonics

The Spherical FNO is an advanced architecture for global atmospheric modeling using spherical harmonic transforms.

### Key Concepts

1. **Spherical Harmonics**: Natural basis for global atmospheric data
2. **Fourier Modes**: Learn in spectral space, transform back to physical space
3. **Multi-scale**: Capture both large-scale (planetary waves) and small-scale (convection) features

### References

- **FourCastNet** (NVIDIA): https://arxiv.org/abs/2202.11214
- **Spherical FNO**: https://arxiv.org/abs/2306.03838
- **torch-harmonics**: https://github.com/NVIDIA/torch-harmonics
- **ClimateLearn**: https://github.com/aditya-grover/climate-learn

### When to Use SFNO

- **Global data**: Working with full atmospheric fields (not just regional subsets)
- **Large-scale patterns**: Need to capture planetary waves and teleconnections
- **Lat/lon grids**: Data is on equiangular or Gaussian grids
- **High performance**: Have GPUs and need efficient spectral transforms

### Implementation

We use NVIDIA's `torch-harmonics` library for efficient spherical harmonic transforms. The library provides:
- Fast SHT with cuFFT acceleration
- Distributed transforms for model parallelism
- DISCO convolutions on the sphere
- Spherical attention mechanisms

In [ ]:
if TORCH_HARMONICS_AVAILABLE:
    # Production SFNO implementation using torch-harmonics
    
    class SphericalFourierLayer(nn.Module):
        """Spherical Fourier layer using torch-harmonics SHT
        
        This layer:
        1. Transforms input from physical space to spectral space (SHT)
        2. Applies learnable spectral convolution
        3. Transforms back to physical space (inverse SHT)
        4. Adds skip connection with MLP
        """
        nlat: int
        nlon: int
        modes_lat: int = 16  # Number of modes in latitude
        modes_lon: int = 16  # Number of modes in longitude
        hidden_dim: int = 256
        grid: str = "equiangular"  # or "legendre-gauss"
        
        def setup(self):
            # Note: SHT in torch-harmonics works with PyTorch tensors
            # For JAX integration, we'll need to use jax.dlpack or convert
            # This is a conceptual implementation showing the structure
            
            # Spectral weights (learnable in spectral space)
            self.spectral_weights_real = self.param(
                'spectral_weights_real',
                nn.initializers.xavier_uniform(),
                (self.modes_lat, self.modes_lon, self.hidden_dim, self.hidden_dim)
            )
            self.spectral_weights_imag = self.param(
                'spectral_weights_imag',
                nn.initializers.xavier_uniform(),
                (self.modes_lat, self.modes_lon, self.hidden_dim, self.hidden_dim)
            )
            
            # Skip connection MLP
            self.mlp = nn.Sequential([
                nn.Dense(self.hidden_dim),
                nn.gelu,
                nn.Dense(self.hidden_dim)
            ])
        
        @nn.compact
        def __call__(self, x):
            """
            x: (batch, lat, lon, channels) in physical space
            returns: (batch, lat, lon, channels) in physical space
            """
            batch, nlat, nlon, channels = x.shape
            residual = x
            
            # NOTE: For actual torch-harmonics integration:
            # 1. Convert JAX array to PyTorch tensor
            # 2. Apply RealSHT
            # 3. Multiply by spectral weights
            # 4. Apply InverseRealSHT
            # 5. Convert back to JAX array
            
            # Simplified version using FFT2D as approximation
            # (Real SFNO would use proper spherical harmonics)
            x_freq = jnp.fft.rfft2(x, axes=(1, 2))
            
            # Truncate to keep only low-frequency modes
            x_freq_truncated = x_freq[:, :self.modes_lat, :self.modes_lon, :]
            
            # Apply spectral convolution (matrix multiply in spectral space)
            # This is where we learn in Fourier domain
            spectral_conv = jnp.einsum(
                'blmc,lmcd->blmd',
                x_freq_truncated,
                self.spectral_weights_real + 1j * self.spectral_weights_imag
            )
            
            # Pad back to original size
            x_freq_out = jnp.zeros_like(x_freq)
            x_freq_out = x_freq_out.at[:, :self.modes_lat, :self.modes_lon, :].set(spectral_conv)
            
            # Inverse FFT to get back to physical space
            x_out = jnp.fft.irfft2(x_freq_out, s=(nlat, nlon), axes=(1, 2))
            
            # Add skip connection with MLP
            x_skip = self.mlp(residual)
            
            return x_out + x_skip
    
    
    class SFNOClimateEmulator(nn.Module):
        """Spherical Fourier Neural Operator for Climate Emulation
        
        Architecture inspired by NVIDIA's FourCastNet and SFNO paper.
        Uses spherical harmonic transforms for efficient learning on the sphere.
        """
        nlat: int = 64  # Latitude grid points
        nlon: int = 128  # Longitude grid points (typically 2*nlat)
        input_channels: int = 5  # Input variables
        output_channels: int = 3  # Output tendencies
        hidden_dim: int = 256
        num_layers: int = 4
        modes_lat: int = 32  # Spectral modes (latitude)
        modes_lon: int = 32  # Spectral modes (longitude)
        dropout_rate: float = 0.1
        grid: str = "equiangular"
        
        @nn.compact
        def __call__(self, x, training: bool = False):
            """
            x: (batch, nlat, nlon, input_channels) - atmospheric state on sphere
            returns: (batch, nlat, nlon, output_channels) - physics tendencies
            """
            
            # Lift: embed to higher dimension
            x = nn.Dense(self.hidden_dim, name='lift')(x)
            x = nn.gelu(x)
            
            # Spherical Fourier layers
            for i in range(self.num_layers):
                x = SphericalFourierLayer(
                    nlat=self.nlat,
                    nlon=self.nlon,
                    modes_lat=self.modes_lat,
                    modes_lon=self.modes_lon,
                    hidden_dim=self.hidden_dim,
                    grid=self.grid,
                    name=f'sfno_layer_{i}'
                )(x)
                x = nn.gelu(x)
                x = nn.Dropout(rate=self.dropout_rate, deterministic=not training)(x)
            
            # Project: map back to output space
            x = nn.Dense(self.output_channels, name='project')(x)
            
            return x, None  # No stochastic output for now
    
    
    print("\n✓ SFNO architecture defined using torch-harmonics structure")
    print("  Note: This uses FFT2D approximation. For production, use actual SHT from torch-harmonics")
    print("  See torch-harmonics examples: https://github.com/NVIDIA/torch-harmonics/tree/main/examples")
    
else:
    # Fallback if torch-harmonics is not available
    print("\n⚠ SFNO not available - torch-harmonics not installed")
    print("  To enable SFNO, run: pip install torch-harmonics")
    print("  Then restart the kernel")

## SFNO Usage Example

To use SFNO instead of 1D CNN, you'll need to:

1. **Install torch-harmonics**: `pip install torch-harmonics`
2. **Prepare data as 2D grid**: Reshape your data to (batch, lat, lon, channels) format
3. **Set model type**: Change `config.MODEL_TYPE = "sfno"`
4. **Adjust grid size**: Ensure lat/lon dimensions match your data

**Note**: For ClimSim NYC subset, the data is columnar (vertical profiles) rather than spatial 2D grids, so **1D CNN is more appropriate**. SFNO is best for global atmospheric data with explicit lat/lon spatial structure.

In [ ]:
# Example: Initialize SFNO model (if available and data is on 2D grid)

if TORCH_HARMONICS_AVAILABLE and config.MODEL_TYPE == "sfno":
    print("Initializing SFNO model...")
    
    # For SFNO, we need 2D spatial data
    # Example dimensions for a small test
    nlat_sfno = 32
    nlon_sfno = 64
    
    sfno_model = SFNOClimateEmulator(
        nlat=nlat_sfno,
        nlon=nlon_sfno,
        input_channels=n_input_vars,
        output_channels=n_output_vars,
        hidden_dim=config.SFNO_WIDTH,
        num_layers=config.SFNO_LAYERS,
        modes_lat=config.SFNO_MODES,
        modes_lon=config.SFNO_MODES,
        dropout_rate=config.DROPOUT_RATE,
        grid=config.grid if hasattr(config, 'grid') else 'equiangular'
    )
    
    # Initialize with dummy 2D spatial data
    rng_sfno = random.PRNGKey(43)
    sample_2d = jnp.ones((1, nlat_sfno, nlon_sfno, n_input_vars))
    sfno_vars = sfno_model.init({'params': rng_sfno, 'dropout': rng_sfno}, sample_2d, training=False)
    sfno_params = sfno_vars['params']
    
    sfno_param_count = sum(x.size for x in jax.tree_util.tree_leaves(sfno_params))
    
    print(f"\nSFNO Model Initialized:")
    print(f"  Grid: {nlat_sfno}×{nlon_sfno}")
    print(f"  Parameters: {sfno_param_count:,}")
    print(f"  Spectral modes: {config.SFNO_MODES}")
    print(f"  Hidden dim: {config.SFNO_WIDTH}")
    
    # Test forward pass
    output_2d, _ = sfno_model.apply(sfno_vars, sample_2d, training=False, rngs={'dropout': rng_sfno})
    print(f"  Output shape: {output_2d.shape}")
    
    print("\n⚠ Note: To actually train SFNO, you need data in (batch, lat, lon, channels) format")
    print("  Current ClimSim data is columnar (vertical profiles), so 1D CNN is more suitable")
    
elif config.MODEL_TYPE == "sfno" and not TORCH_HARMONICS_AVAILABLE:
    print("⚠ Cannot initialize SFNO: torch-harmonics not installed")
    print("  Install with: pip install torch-harmonics")
else:
    print(f"Using {config.MODEL_TYPE.upper()} architecture (as configured)")

## JAX-PyTorch Interoperability for torch-harmonics

If you want to use actual torch-harmonics SHT in a JAX workflow, you can bridge the frameworks using `jax.dlpack` and `torch.utils.dlpack`. Here's a helper function:

In [ ]:
if TORCH_HARMONICS_AVAILABLE:
    from jax import dlpack as jax_dlpack
    from torch.utils import dlpack as torch_dlpack
    
    def jax_to_torch(x_jax):
        """Convert JAX array to PyTorch tensor via DLPack (zero-copy)"""
        x_dlpack = jax_dlpack.to_dlpack(x_jax)
        x_torch = torch_dlpack.from_dlpack(x_dlpack)
        return x_torch
    
    def torch_to_jax(x_torch):
        """Convert PyTorch tensor to JAX array via DLPack (zero-copy)"""
        x_dlpack = torch_dlpack.to_dlpack(x_torch)
        x_jax = jax_dlpack.from_dlpack(x_dlpack)
        return x_jax
    
    def apply_torch_sht_in_jax(x_jax, nlat, nlon, grid="equiangular"):
        """
        Apply torch-harmonics SHT within a JAX workflow
        
        Args:
            x_jax: JAX array of shape (batch, nlat, nlon, channels)
            nlat: Number of latitude points
            nlon: Number of longitude points
            grid: "equiangular" or "legendre-gauss"
        
        Returns:
            Spectral coefficients as JAX array
        """
        # Convert to PyTorch
        x_torch = jax_to_torch(x_jax)
        
        # Move to GPU if available
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        x_torch = x_torch.to(device)
        
        # Create SHT operator
        sht = th.RealSHT(nlat, nlon, grid=grid).to(device)
        
        # Apply transform (over last 2 dimensions)
        # torch-harmonics expects (batch, channels, nlat, nlon)
        x_torch = x_torch.permute(0, 3, 1, 2)
        coeffs_torch = sht(x_torch)
        
        # Convert back to JAX
        coeffs_jax = torch_to_jax(coeffs_torch)
        
        return coeffs_jax
    
    print("✓ JAX-PyTorch interop functions defined")
    print("  Use these to integrate torch-harmonics SHT in JAX training loops")
    print("  Note: This may have performance overhead due to framework conversion")
    
    # Example usage
    if config.MODEL_TYPE == "sfno":
        example_2d = jnp.ones((2, 32, 64, 3))  # Small test
        print(f"\nExample: Applying SHT to shape {example_2d.shape}")
        try:
            coeffs_example = apply_torch_sht_in_jax(example_2d, nlat=32, nlon=64)
            print(f"  Spectral coefficients shape: {coeffs_example.shape}")
        except Exception as e:
            print(f"  (Would work with GPU/proper setup, got: {type(e).__name__})")
else:
    print("JAX-PyTorch interop not available (torch-harmonics not installed)")

## Summary & Next Steps

In [ ]:
print(f"\n{'='*80}")
print("ADVANCED CLIMATE EMULATOR - SUMMARY")
print(f"{'='*80}\n")

print("✅ What We Built:\n")
print("  1. 1D CNN Architecture")
print(f"     - {len(config.CNN_CHANNELS)} residual blocks with {config.CNN_CHANNELS} channels")
print(f"     - Group normalization, dropout, swish activation")
print(f"     - Parameters: {param_count:,}\n")

print("  2. Stochastic Output Head")
print(f"     - Predicts mean and uncertainty (std) for each output")
print(f"     - Enables probabilistic forecasting and risk assessment\n")

print("  3. Per-Variable-Group Monitoring")
print(f"     - R² tracked separately for temperature, moisture, clouds")
print(f"     - Final R² scores:")
print(f"       * Temperature:  {test_r2_metrics['r2_temperature']:.4f}")
print(f"       * Moisture:     {test_r2_metrics['r2_moisture']:.4f}")
print(f"       * Clouds:       {test_r2_metrics['r2_clouds']:.4f}")
print(f"       * Overall:      {test_r2_metrics['r2_overall']:.4f}\n")

if config.USE_PMAP:
    print("  4. Multi-GPU Training (pmap)")
    print(f"     - Data parallel across {jax.device_count()} devices")
    print(f"     - Effective batch size: {config.BATCH_SIZE * jax.device_count()}\n")

print(f"\n📊 Performance Metrics:\n")
print(f"  Test MSE:    {test_mse:.6f}")
print(f"  Test MAE:    {test_mae:.6f}")
print(f"  Test RMSE:   {test_rmse:.6f}")
print(f"  Test R²:     {test_r2_metrics['r2_overall']:.4f}")
if config.STOCHASTIC_OUTPUT:
    print(f"  Mean Std:    {mean_uncertainty:.6f}")

print(f"\n💾 Saved Artifacts:\n")
print(f"  {config.MODEL_DIR / 'best_model' / 'checkpoint'}")
print(f"  {config.MODEL_DIR / 'training_results.npz'}")
print(f"  {config.MODEL_DIR / 'training_curves.png'}")
print(f"  {config.MODEL_DIR / 'vertical_profiles.png'}")
print(f"  {config.MODEL_DIR / 'prediction_scatter.png'}")

print(f"\n🚀 Next Steps:\n")
print("  1. **Hyperparameter Tuning**")
print("     - Try different channel progressions")
print("     - Experiment with kernel sizes (3, 5, 7)")
print("     - Adjust dropout and weight decay\n")

print("  2. **Advanced Architectures**")
print("     - Implement U-Net style skip connections")
print("     - Add attention mechanisms")
print("     - Explore Spherical FNO for global modeling\n")

print("  3. **Physics Constraints**")
print("     - Enable water conservation loss")
print("     - Add energy conservation constraints")
print("     - Enforce physical bounds (e.g., humidity > 0)\n")

print("  4. **Ensemble Methods**")
print("     - Train multiple models with different seeds")
print("     - Use stochastic head for ensemble generation")
print("     - Calibrate uncertainty estimates\n")

print("  5. **Scaling to Full Dataset**")
print("     - Train on full ClimSim low-res (not just NYC)")
print("     - Use multi-node training if available")
print("     - Implement gradient checkpointing for memory efficiency\n")

print("  6. **Evaluation & Analysis**")
print("     - Analyze per-level performance")
print("     - Test on extreme events")
print("     - Compare with physics-based parameterizations\n")

print(f"{'='*80}")
print("🌍 Your advanced climate emulator is ready for experimentation! 🚀")
print(f"{'='*80}\n")